In [33]:
import numpy as np

## 1. Products are sourced independently

In [34]:
demands = np.array([1000, 300, 100, 50])
common_ordering_cost = 100
specific_ordering_costs = np.array([10, 20, 25, 25])
unit_costs = np.array([50, 60, 30, 30])
holding_cost_rates = np.array([0.2, 0.2, 0.2, 0.2])

# Calculate EOQ for each product
eoqs = np.sqrt((2 * demands * (common_ordering_cost + specific_ordering_costs)) / (holding_cost_rates * unit_costs))

# Calculate total costs for each product when sourced independently
ordering_costs_independent = (demands / eoqs) * (common_ordering_cost + specific_ordering_costs)
holding_costs_independent = (eoqs / 2) * holding_cost_rates * unit_costs
total_costs_independent = ordering_costs_independent + holding_costs_independent

# Sum of total costs for independent sourcing
total_annual_cost = total_costs_independent.sum()

In [35]:
products = ["Product 1", "Product 2", "Product 3", "Product 4"]

for i in range(4):
    print(f"{products[i]}:")
    print(f"EOQ: {eoqs[i]:.2f} units")
    print(f"Ordering Cost: ${ordering_costs_independent[i]:.2f}")
    print(f"Holding Cost: ${holding_costs_independent[i]:.2f}")
    print(f"Total Cost: ${total_costs_independent[i]:.2f}\n")

print(f"Total Annual Operational Cost: ${total_annual_cost:.2f}")

Product 1:
EOQ: 148.32 units
Ordering Cost: $741.62
Holding Cost: $741.62
Total Cost: $1483.24

Product 2:
EOQ: 77.46 units
Ordering Cost: $464.76
Holding Cost: $464.76
Total Cost: $929.52

Product 3:
EOQ: 64.55 units
Ordering Cost: $193.65
Holding Cost: $193.65
Total Cost: $387.30

Product 4:
EOQ: 45.64 units
Ordering Cost: $136.93
Holding Cost: $136.93
Total Cost: $273.86

Total Annual Operational Cost: $3073.92


## 2. All four products are sourced with the same frequency

In [36]:
import math

# Calculations
transaction_cost = common_ordering_cost + specific_ordering_costs.sum()
total_demand_holding_cost = np.sum(demands * holding_cost_rates * unit_costs)
optimal_frequency = np.sqrt(total_demand_holding_cost / (2 * transaction_cost))
optimal_order_sizes = np.round(demands / optimal_frequency, 0)
annual_holding_costs = optimal_order_sizes / 2 * holding_cost_rates * unit_costs
annual_ordering_cost = optimal_frequency * transaction_cost
total_annual_cost_2 = annual_holding_costs.sum() + annual_ordering_cost

# Output
for i, (name, eoq, holding_costs_independent) in enumerate(zip(products, optimal_order_sizes, annual_holding_costs), 1):
    ordering_cost_independent = optimal_frequency * (common_ordering_cost + specific_ordering_costs[i-1])
    total_cost_independent = holding_costs_independent + ordering_cost_independent
    print(f"{name}:")
    print(f"EOQ: {eoq} units")
    print(f"Ordering Cost: ${ordering_cost_independent:.2f}")
    print(f"Holding Cost: ${holding_costs_independent:.2f}")
    print(f"Total Cost: ${total_cost_independent:.2f}\n")

print(f"Total Annual Operational Cost: ${total_annual_cost_2:.2f}")

Product 1:
EOQ: 158.0 units
Ordering Cost: $698.11
Holding Cost: $790.00
Total Cost: $1488.11

Product 2:
EOQ: 47.0 units
Ordering Cost: $761.58
Holding Cost: $282.00
Total Cost: $1043.58

Product 3:
EOQ: 16.0 units
Ordering Cost: $793.31
Holding Cost: $48.00
Total Cost: $841.31

Product 4:
EOQ: 8.0 units
Ordering Cost: $793.31
Holding Cost: $24.00
Total Cost: $817.31

Total Annual Operational Cost: $2286.37


## 3. Order frequencies are determined according to the tailored aggregation strategy

In [37]:
# Calculating m-values based on frequency comparison to Product 1
frequency_product1 = 6.7568
ms = [1] + [math.ceil(frequency_product1 / (d / np.sqrt((2 * d * s) / (h * u)))) for d, s, h, u in zip(demands[1:], specific_ordering_costs[1:], holding_cost_rates[1:], unit_costs[1:])]

# Adjusted ordering costs including common cost for Product 1 and calculating adjusted frequency
specific_ordering_costs_adjusted = np.array([common_ordering_cost + specific_ordering_costs[0]] + list(specific_ordering_costs[1:]))
optimal_frequency_adjusted = round(math.sqrt(sum(d * h * m for d, h, m in zip(demands, holding_cost_rates * unit_costs, ms)) / (2 * (common_ordering_cost + sum(s / m for s, m in zip(specific_ordering_costs[1:], ms[1:]))))), 1)

# Optimal order quantities and frequencies for all products
optimal_order_quantities_3 = [round(d / (optimal_frequency_adjusted if i == 0 else optimal_frequency_adjusted / ms[i]), 0) for i, d in enumerate(demands)]

# Calculating costs and total annual operational cost
annual_ordering_cost_3 = sum(s * f for s, f in zip(specific_ordering_costs_adjusted, [optimal_frequency_adjusted] + [optimal_frequency_adjusted / m for m in ms[1:]]))
annual_holding_cost_3 = sum(q / 2 * h * u for q, h, u in zip(optimal_order_quantities_3, holding_cost_rates, unit_costs))
annual_total_cost_3 = annual_ordering_cost_3 + annual_holding_cost_3

# Output the results
for i, name in enumerate(products):
    individual_ordering_cost = specific_ordering_costs_adjusted[i] * (optimal_frequency_adjusted if i == 0 else optimal_frequency_adjusted / ms[i])
    individual_holding_cost = optimal_order_quantities_3[i] / 2 * holding_cost_rates[i] * unit_costs[i]
    print(f"{name}:")
    print(f"EOQ: {optimal_order_quantities_3[i]} units")
    print(f"Ordering Cost: ${individual_ordering_cost:.2f}")
    print(f"Holding Cost: ${individual_holding_cost:.2f}")
    print(f"Total Cost: ${individual_ordering_cost + individual_holding_cost:.2f}\n")

print(f"Total Annual Operational Cost: ${annual_total_cost_3:.2f}")

Product 1:
EOQ: 133.0 units
Ordering Cost: $825.00
Holding Cost: $665.00
Total Cost: $1490.00

Product 2:
EOQ: 40.0 units
Ordering Cost: $150.00
Holding Cost: $240.00
Total Cost: $390.00

Product 3:
EOQ: 27.0 units
Ordering Cost: $93.75
Holding Cost: $81.00
Total Cost: $174.75

Product 4:
EOQ: 20.0 units
Ordering Cost: $62.50
Holding Cost: $60.00
Total Cost: $122.50

Total Annual Operational Cost: $2177.25
